In [7]:
import pandas as pd
import numpy as np
from travel_distance_map import VBBAPICached, Query, GPSPoint, Position
from datetime import datetime, timedelta

# Compute travel time for an equally spaced rectangular area

In [8]:
with open('ACCESS_ID.txt') as f:
    ACCESS_ID = f.read().strip()

In [9]:
api = VBBAPICached(ACCESS_ID)

In [10]:
# topleft = GPSPoint(52.522, 13.4322)
# bottomright = GPSPoint(52.5005, 13.4727)
topleft = GPSPoint(52.5307, 13.4082)
bottomright = GPSPoint(52.4999, 13.5005)
num_points = 20
dest = Position(52.5219216, 13.411026, 900100003, 'Alexanderplatz')
WALK_VELOCITY = 5.0/3.6
dt = datetime(2020, 6, 10, 5, 0, 0)

Compute list of nearest stops and then compute travel time for them, then interpolate between the stops using distance transform and walking speed

In [11]:
df = pd.DataFrame(columns=['latitude', 'longitude'])

lat_arr = np.linspace(topleft.lat, bottomright.lat, num_points)
lon_arr = np.linspace(topleft.lon, bottomright.lon, num_points)
stops = []
index = 0
for lat in lat_arr:
    for lon in lon_arr:
        index += 1
        origin = GPSPoint(lat, lon)
        # closest stop
        closest_stop = api.get_closest_stop(origin)
        stops.append(closest_stop)
        print(('s', 'S')[index%10==0], end='')
print()
        
for index, stop in enumerate(stops):
    # print(stop)
    trips = api.get_all_trips(stop, dest, dt)
    sorted_trips = sorted(trips, key=lambda x:x[0])

    if sorted_trips:
        shortest_trip = sorted_trips[0]
        shortest_trip_time = shortest_trip[0].total_seconds()
        success = True
    else:
        shortest_trip = (origin.distance_utf(dest.point)/WALK_VELOCITY, ['No trip found'])
        shortest_trip_time = origin.distance_utf(dest.point)/WALK_VELOCITY
        success = False
    shortest_trip_itinerary = shortest_trip[1]
        
    df = df.append({'latitude': stop.point.lat, 
                    'longitude': stop.point.lon, 
                    'ID': stop.id, 
                    'name': stop.name,
                    'shortest travel time': shortest_trip_time,
                    'shrotest trip itinerary': shortest_trip_itinerary
                   }, ignore_index=True)
    print(('.', 'X')[index%10==0], end='')

sssssssssSsssssssssSsssssssssSsssssssssSsssssssssSsssssssssSsssssssssSsssssssssSsssssssssSsssssssssSsssssssssSsssssssssSsssssssssSsssssssssSsssssssssSsssssssssSsssssssssSsssssssssSsssssssssSsssssssssSsssssssssSsssssssssSsssssssssSsssssssssSsssssssssSsssssssssSsssssssssSsssssssssSsssssssssSsssssssssSsssssssssSsssssssssSsssssssssSsssssssssSsssssssssSsssssssssSsssssssssSsssssssssSsssssssssSsssssssssS
X.........X.........X.........X.........X.........X.........X.........X.........X.........X.........X.........X.........X.........X.........X.........X.........X.........X.........X.........X.........X.........X.........X.........X.........X.........X.........X.........X.........X.........X.........X.........X.........X.........X.........X.........X.........X.........X.........X.........X.........

In [12]:
df

,latitude,longitude,ID,name,shortest travel time,shrotest trip itinerary
0,52.528187,13.410405,900100016,U Rosa-Luxemburg-Platz (Berlin),120.0,"[U Rosa-Luxemburg-Platz (Berlin), S+U Alexande..."
1,52.532619,13.412625,900110005,U Senefelderplatz (Berlin),180.0,"[U Senefelderplatz (Berlin), S+U Alexanderplat..."
2,52.531028,13.419385,900110506,Prenzlauer Allee/Metzer Str. (Berlin),180.0,"[Prenzlauer Allee/Metzer Str. (Berlin), S+U Al..."
3,52.531028,13.419385,900110506,Prenzlauer Allee/Metzer Str. (Berlin),180.0,"[Prenzlauer Allee/Metzer Str. (Berlin), S+U Al..."
4,52.532520,13.428787,900110521,Hufelandstr. (Berlin),360.0,"[Hufelandstr. (Berlin), U Alexanderplatz (Berl..."
...,...,...,...,...,...,...
395,52.501211,13.478696,900160001,S Rummelsburg (Berlin),600.0,"[S Rummelsburg (Berlin), S+U Warschauer Str. (..."
396,52.502271,13.485141,900160534,S Nöldnerplatz/Schlichtallee (Berlin),960.0,"[S Nöldnerplatz/Schlichtallee (Berlin), S Nöld..."
397,52.501705,13.493447,900160019,Lückstr./Weitlingstr. (Berlin),780.0,"[Lückstr./Weitlingstr. (Berlin), S Rummelsburg..."
398,52.500267,13.496108,900160519,Wönnichstr. (Berlin),720.0,"[Wönnichstr. (Berlin), S Nöldnerplatz (Berlin)..."


Create a dense interpolated map

In [13]:
points = df[['latitude','longitude']].to_numpy()
values = df[['shortest travel time']].to_numpy().reshape((len(df), ))

In [14]:
NUM_INTERPOLATED_POINTS = 100j
grid_x, grid_y = np.mgrid[bottomright.lat:topleft.lat:NUM_INTERPOLATED_POINTS, topleft.lon:bottomright.lon:NUM_INTERPOLATED_POINTS]

In [15]:
bottomleft = GPSPoint(topleft.lat, bottomright.lon)
dist_y = topleft.distance_utf(bottomleft)

topright = GPSPoint(bottomright.lat, topleft.lon)
dist_x = topleft.distance_utf(topright)

distance_between_adjacent_points = np.asarray((np.abs(dist_x/NUM_INTERPOLATED_POINTS/WALK_VELOCITY), np.abs(dist_y/NUM_INTERPOLATED_POINTS/WALK_VELOCITY)))

In [16]:
def interpolate(points, values, grid, distance_fun):
    """
    Rasterizes data points on a grid and interpolates between the points using distance transformation function.
    
    @param points list of 2D points
    @param values list of 1D point values
    @param grid tuple of grid index arrays (grid_x, grid_y), cmp. numpy.mgrid class
    @param distance_fun distance function taking two points as arguments and returning their distance value
    @return distance map
    """
    grid_x, grid_y = grid
    distance_map = np.zeros_like(grid_x)
    shape = grid_x.shape
    assert grid_x.shape == grid_y.shape
    for col_index in range(shape[1]):
        for row_index in range(shape[0]):
            value_x = grid_x[row_index, col_index]
            value_y = grid_y[row_index, col_index]
            distance_map[row_index, col_index] = min(distance_fun(GPSPoint(value_x, value_y), GPSPoint(point[0], point[1]))+value for point, value in zip(points, values))
    return distance_map

def dist_utf(point1, point2):
    return point1.distance_utf(point2)/WALK_VELOCITY
# def eucl_dist_fun(point1, point2):
#     return np.linalg.norm(point1-point2)
distance_map = interpolate(points, values, (grid_x,grid_y), dist_utf)

In [17]:
map_center = (topleft.lat + bottomright.lat)/2.0, (topleft.lon + bottomright.lon)/2.0

Render results as map overlay

In [18]:
import folium
from folium import plugins
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

%matplotlib inline

In [19]:
top = cm.get_cmap('Oranges_r', 128)
bottom = cm.get_cmap('Greens', 128)
newcolors = np.vstack((top(np.linspace(0.2, 0.8, 128)),
                       bottom(np.linspace(0.2, 0.8, 128))))
newcmp = ListedColormap(newcolors, name='OrangeBlue')

In [20]:
m = folium.Map(map_center, zoom_start=14)
colors = ['green', 'orange', 'red']
for index, row in df.iterrows():
    tt = row['shortest travel time']
    color = colors[min(int(tt//500), len(colors))]
    folium.CircleMarker((row['latitude'], row['longitude']), fill_color=color, opacity=0.8, radius=3, color=color, stroke=True, tooltip='{:.2f}'.format(row['shortest travel time'])).add_to(m)
    
overlay = distance_map
norm_factor = np.percentile(values,95)
norm_fun = np.vectorize(lambda x: 1 - min(x/norm_factor, 1))
overlay = norm_fun(overlay)

[min_lat, min_lon] = bottomright.lat, topleft.lon
[max_lat, max_lon] = topleft.lat, bottomright.lon

folium.raster_layers.ImageOverlay(
    image=overlay,
    bounds=[[min_lat, min_lon], [max_lat, max_lon]],
    colormap=newcmp,
    vmin=0,
    vmax=3600,
    opacity=0.7,
    pixelated=False,
    origin='lower'
).add_to(m)

# enable for debugging
# shape = grid_x.shape
# for col_index in range(0,shape[1],5):
#     for row_index in range(0,shape[0],5):
#         point = GPSPoint(grid_x[col_index][row_index], grid_y[col_index][row_index])
#         distance_map_value = distance_map[col_index][row_index]
#         folium.CircleMarker((point.lat, point.lon), fill_color='black', opacity=0.8, radius=3, color='black', stroke=True, tooltip=distance_map_value).add_to(m)

m.save('./distance_map.html')

Geocoding example

In [21]:
import urllib.parse
import requests

In [22]:
address = "Frankfurter Allee 15, 10247"
address = "Pfuelstraße 1"
query = urllib.parse.quote_plus(address)
url = "https://nominatim.openstreetmap.org/?addressdetails=1&q={}&format=json&limit=1".format(query)
response = requests.get(url)

In [23]:
js = response.json()
geo_point = GPSPoint(float(js[0]['lat']), float(js[0]['lon']))

In [24]:
def get_closest_grid_point(grid, point):
    grid_x, grid_y = grid
    search_arr = (grid_x - point.lat)**2 + (grid_y - point.lon)**2
    min_element = np.argmin(np.abs(search_arr))
    return min_element

def get_distance_for_geopoint(grid, point, distance_map):
    index = get_closest_grid_point((grid_x, grid_y), point)
    return distance_map.ravel()[index]

In [25]:
get_distance_for_geopoint((grid_x,grid_y), geo_point, distance_map)

688.0163669900921